In [ ]:
# Dependencies and setup
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter("ignore")

# Read the CSV and Perform Basic Data Cleaning

In [ ]:
# Read CSV into DataFrame
df = pd.read_csv("../Data/exoplanet_data.csv")

# Drop the null columns where all values are null
df = df.dropna(axis = "columns", how = "all")

# Drop the null rows
df = df.dropna()

# Display DataFrame
df.head()

In [ ]:
# List all columns
df.columns

# Select your features (columns)

In [ ]:
# Set features to be used as x values
selected = df.drop(columns = ["koi_disposition"])
X = selected
X.head()

# Create a Train Test Split

Use `koi_disposition` for the y values

In [ ]:
# Set y values
y = df["koi_disposition"].values.reshape(-1, 1)
print(X.shape, y.shape)

In [ ]:
# Dependencies and setup
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)
X_train.head()

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [ ]:
# Dependencies and setup
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

# Scale data
X_scale = MinMaxScaler().fit(X_train)
X_train_scaled = X_scale.transform(X_train)
X_test_scaled = X_scale.transform(X_test)

# Label Encode dataset
labelEncoder = LabelEncoder()
labelEncoder.fit(y_train)
y_train = labelEncoder.transform(y_train)
y_test = labelEncoder.transform(y_test)

# Train the Model

In [ ]:
# Dependencies and setup
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt

# Loop through different k values to see which has the highest accuracy
train_scores = []
test_scores = []
for k in range(1, 20, 2):
    knn = KNeighborsClassifier(n_neighbors = k)
    knn.fit(X_train_scaled, y_train)
    train_score = knn.score(X_train_scaled, y_train)
    test_score = knn.score(X_test_scaled, y_test)
    train_scores.append(train_score)
    test_scores.append(test_score)
    
    # Validate the model using test data
    print(f"k: {k}, Train/Test Score: {train_score:.3f}/{test_score:.3f}")

In [ ]:
# Plot scores and k neighbors
plt.plot(range(1, 20, 2), train_scores, marker = "o")
plt.plot(range(1, 20, 2), test_scores, marker = "x")
plt.xlabel("k neighbors")
plt.ylabel("Testing accuracy Score")

# Display plot
plt.show()

In [ ]:
# k value of 11 provides the best accuracy where the classifier starts to stabilize
knn = KNeighborsClassifier(n_neighbors = 11)
knn.fit(X_train_scaled, y_train)
print("k = 11 Test Acc: %.3f" % knn.score(X_test_scaled, y_test))

In [ ]:
# Make predictions
predictions = knn.predict(X_test_scaled)
print(f"First 10 Predictions:   {predictions[:10]}")
print(f"First 10 Actual labels: {y_test[:10].tolist()}")

In [ ]:
# Dependencies and setup
from sklearn.metrics import classification_report

# Calculate classification report
print(classification_report(y_test, predictions, target_names = ["Confirmed", "Candidate", "False Positive"]))

# Save the Model

In [ ]:
# Dependencies and setup
import joblib

# Save the model
filename = "../Models/KNN.sav"
joblib.dump(knn, filename)